<a href="https://colab.research.google.com/github/qianzhou1982/Demo/blob/master/Cr%C3%A9ation_d'ILC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interface en ligne de commande pour GitHub Issues

Le but de ces travaux pratiques est d'utiliser un maximum de notions vues durant la formation. Nous allons créer une interface en ligne de commande pour GitHub Issues, créer un paquet et le publier sur Test PyPI.

## Installation de `poetry`

In [ ]:
!rm -rf sample_data
!curl -sSL https://raw.githubusercontent.com/python-poetry/poetry/master/get-poetry.py | python -
path = %env PATH
poetry_path = "/root/.poetry/bin"
if poetry_path not in path:
  path = f"{poetry_path}:{path}"
  %set_env PATH $path
!poetry config repositories.testpypi https://test.pypi.org/legacy/
!poetry config virtualenvs.create false

## Création du projet avec `poetry`

Utilisez `poetry` pour créer un projet Python, appelé `github_cli_{pseudopypi}`.

In [ ]:
%%bash
poetry new ghcli-m09

Corrigez ici le chemin vers votre projet, cela permettra d'exécuter directement toutes les commandes par la suite.

In [ ]:
%cd /content/ghcli-m09

## Définition d'une ILC

Pour ces travaux pratiques, on souhaite implémenter deux fonctionnalités :
- La création d'une issue (un ticket de bug) automatique dans GitHub
- Le listage des issues existantes dans un projet GitHub

Ces deux fonctionnalités constituent deux commandes naturelles pour une ILC, appelons les `create` et `list`.

La commande `list` devra savoir quel dépôt GitHub utiliser pour aller chercher les issues et prendra donc deux arguments, `owner` et `repo`

La commande `create` devra aussi savoir quel dépôt GitHub utiliser et aura donc aussi des arguments `owner` et `repo`, auxquels il faudra rajouter un argument pour le titre de l'issue à créer, `title` et un argument pour le corps de l'issue à créer, `body`.

Créez un fichier `cli.py` dans votre projet et utilisez `argparse` pour définir une ILC comme décrite ci-dessus.

## Implémentation de `list` et `create`

Créez un fichier `api.py` dans votre paquet qui contiendra la logique dont les deux commandes se serviront. Pour faire une requête sur l'API GitHub, on peut utiliser l'excellent module `requests` (installez le maintenant avec `poetry`). Par exemple, pour `list`, l'apppel ressemblerait à :

Vous noterez qu'il faut un token pour que l'API GitHub fonctionne. Vous pouvez le créer ici : https://github.com/settings/tokens.

Il faut donner l'autorisation `public_repo` pour pouvoir créer des issues sur des dépôts publics.

```python
response = requests.get(
    f"https://api.github.com/repos/{owner}/{repo}/issues",
    headers=dict(
        Accept="application/vnd.github.v3+json",
        Authorization=f"token {_GITHUB_TOKEN}",
    ),
)
```
Pour `create`, il ressemblerait à :
```python
requests.post(
    f"https://api.github.com/repos/{owner}/{repo}/issues",
    json=data,
    headers=dict(
        Accept="application/vnd.github.v3+json",
        Authorization=f"token {_GITHUB_TOKEN}",
    ),
)
```
où `data` serait un dictionnaire contenant une clef `title` et une clef `body`.

Implémentez à l'aide de ces deux appels les fonctions `create_issue` et `list_issues`.

## Définition du point d'entrée

Pour pouvoir appeler facilement votre programme depuis la ligne de commande, il faut définir un point d'entrée. Pour cela on modifie le fichier `pyproject.toml` de la manière suivante :

```ini
[tool.poetry.scripts]
ghcli = 'github_cli_{pseudopypi}.cli:main'
```

In [ ]:
%%bash
ghcli --help

## Tests

Maintenant que votre projet a des fonctionnalités de base, il est temps d'ajouter des tests (dans une démarche TDD, on aurait bien sûr commencé par ceux-ci).

Ajoutez un test par fonction dans le dossier `tests`.

## Publication

Tout semble prêt pour une première release ! Publiez votre paquet sur Test PyPI (pour rappel, utilisez bien l'argument `-r testpypi` dans votre commande de publication).